# Final Project - Applied Data Science Capstone

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

#### Passing Foursquare credentials

In [ ]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [3]:
address = 'Columbus, OH'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.9622601 -83.0007065


In [4]:
search_query = ''
radius = 500
print(search_query + ' .... OK!')

 .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TRQKRBAK4FVZ3KTQHJY2HSBXIVX55NXRFAFBJF3QGCFWD5AQ&client_secret=RFMBGJTUHDXBTUEJOLF5R2WM5HGJUNP2UIOSCET1FC3KFECI&ll=39.9622601,-83.0007065&v=20180604&query=&radius=500&limit=200'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e28f564ad1ab4001cee2706'},
 'response': {'venues': [{'id': '4e0e63567d8bb178a8a8e0b6',
    'name': 'Tim Hortons',
    'location': {'address': '2 E Broad St',
     'crossStreet': 'North High St',
     'lat': 39.9624524319083,
     'lng': -83.0004241839106,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.9624524319083,
       'lng': -83.0004241839106}],
     'distance': 32,
     'postalCode': '43215',
     'cc': 'US',
     'city': 'Columbus',
     'state': 'OH',
     'country': 'United States',
     'formattedAddress': ['2 E Broad St (North High St)',
      'Columbus, OH 43215',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1579742601',
    'hasPerk': Fa

In [7]:
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe_ColumbusCity = json_normalize(venues)
dataframe_ColumbusCity.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,4e0e63567d8bb178a8a8e0b6,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1579742601,False,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"[2 E Broad St (North High St), Columbus, OH 43...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4dd6c521d22d38ef42d7e74f,Broad & High,"[{'id': '52f2ab2ebcbc57f1066b8b4c', 'name': 'I...",v-1579742601,False,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"[Broad Street (High street), Columbus, OH 4321...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ba25a3ef964a5203bf037e3,Cinco Burritos & Tacos,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",v-1579742601,False,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"[1 S High St (at Broad St.), Columbus, OH 4321...",1186496,https://www.grubhub.com/restaurant/cinco-resta...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
3,4cb8c2e586dd9521a5abaae6,COTA Bus Stop @ Broad & High,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1579742601,False,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"[Broad Street (at High Street), Columbus, OH 4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4caf6efaef1b370464743200,Capitol Square,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1579742601,False,1 Capitol Sq,NaN,39.960905,-82.999606,"[{'label': 'display', 'lat': 39.9609053407906,...",177,43215,US,Columbus,OH,United States,"[1 Capitol Sq, Columbus, OH 43215, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_ColumbusCity.columns if col.startswith('location.')] + ['id']
dataframe_ColumbusCity_filtered = dataframe_ColumbusCity.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_ColumbusCity_filtered['categories'] = dataframe_ColumbusCity_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_ColumbusCity_filtered.columns = [column.split('.')[-1] for column in dataframe_ColumbusCity_filtered.columns]

dataframe_ColumbusCity_filtered.head(50)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"[2 E Broad St (North High St), Columbus, OH 43...",NaN,4e0e63567d8bb178a8a8e0b6
1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"[Broad Street (High street), Columbus, OH 4321...",NaN,4dd6c521d22d38ef42d7e74f
2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"[1 S High St (at Broad St.), Columbus, OH 4321...",NaN,4ba25a3ef964a5203bf037e3
3,COTA Bus Stop @ Broad & High,Bus Station,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"[Broad Street (at High Street), Columbus, OH 4...",NaN,4cb8c2e586dd9521a5abaae6
4,Capitol Square,Park,1 Capitol Sq,NaN,39.960905,-82.999606,"[{'label': 'display', 'lat': 39.9609053407906,...",177,43215,US,Columbus,OH,United States,"[1 Capitol Sq, Columbus, OH 43215, United States]",NaN,4caf6efaef1b370464743200
5,The Daily Growler,Beer Bar,1500 Polaris Pkwy,NaN,40.145624,-82.982058,"[{'label': 'display', 'lat': 40.14562420000001...",20473,43240,US,Columbus,OH,United States,"[1500 Polaris Pkwy, Columbus, OH 43240, United...",NaN,5c76d4841cf2e10039e29680
6,Crowe Horwath LLP,Office,155 W Nationwide Blvd,NaN,39.962236,-83.000652,"[{'label': 'display', 'lat': 39.96223647726225...",5,43215,US,Columbus,OH,United States,"[155 W Nationwide Blvd, Columbus, OH 43215, Un...",NaN,4fe85e7ae4b0117429aa1c41
7,"Hotel LeVeque, Autograph Collection",Hotel,50 W Broad St,Broad and Front,39.962299,-83.002059,"[{'label': 'display', 'lat': 39.96229935639491...",115,43215,US,Columbus,OH,United States,"[50 W Broad St (Broad and Front), Columbus, OH...",NaN,58c96261735a4d6c4d5595a3
8,Huntington Bank,Bank,17 South High Street,NaN,39.961746,-83.000966,"[{'label': 'display', 'lat': 39.96174568, 'lng...",61,43215,US,Columbus,OH,United States,"[17 South High Street, Columbus, OH 43215, Uni...",NaN,4b0aef07f964a5204a2a23e3
9,Jimmy John's,Sandwich Place,20 S Front St,W Capital St,39.961554,-83.002333,"[{'label': 'display', 'lat': 39.96155426911419...",159,43215,US,Columbus,OH,United States,"[20 S Front St (W Capital St), Columbus, OH 43...",NaN,4dc17ee9ae60533f34f34667


## Visualizing veneues on Map

In [9]:

from geopy.geocoders import Nominatim 

address = 'Columbus, OH'
geolocator = Nominatim(user_agent="Columbus, OH")
location = geolocator.geocode(address)
latitude = float(location.latitude)
longitude = float(location.longitude)
print('The geograpical coordinate of Columbus, Ohio are {}, {}.'.format(latitude, longitude))

# create map of all codes using latitude and longitude values

map_Columbus= folium.Map(location=[latitude, longitude], tiles = 'Stamen Terrain', zoom_start=18)

# add markers to map
for lat, lng, borough, neighborhood in zip(dataframe_ColumbusCity_filtered['lat'], dataframe_ColumbusCity_filtered['lng'], dataframe_ColumbusCity_filtered['name'], dataframe_ColumbusCity_filtered['city']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Columbus)  
    
map_Columbus

The geograpical coordinate of Columbus, Ohio are 39.9622601, -83.0007065.


In [10]:

dataframe_ColumbusCity_filtered['name'] = dataframe_ColumbusCity_filtered['name'].astype(str)
dataframe_ColumbusCity_filtered['categories'] = dataframe_ColumbusCity_filtered['categories'].astype(str)
dataframe_ColumbusCity_filtered['address'] = dataframe_ColumbusCity_filtered['address'].astype(str)
dataframe_ColumbusCity_filtered['city'] = dataframe_ColumbusCity_filtered['city'].astype(str)
dataframe_ColumbusCity_filtered['state'] = dataframe_ColumbusCity_filtered['state'].astype(str)
dataframe_ColumbusCity_filtered['country'] = dataframe_ColumbusCity_filtered['country'].astype(str)
dataframe_ColumbusCity_filtered['formattedAddress'] = dataframe_ColumbusCity_filtered['formattedAddress'].astype(str)
dataframe_ColumbusCity_filtered['neighborhood'] = dataframe_ColumbusCity_filtered['neighborhood'].astype(str)
dataframe_ColumbusCity_filtered.columns

Index(['name', 'categories', 'address', 'crossStreet', 'lat', 'lng',
       'labeledLatLngs', 'distance', 'postalCode', 'cc', 'city', 'state',
       'country', 'formattedAddress', 'neighborhood', 'id'],
      dtype='object')

In [11]:
df = dataframe_ColumbusCity_filtered
df['categories']

0                          Coffee Shop
1                         Intersection
2                        Burrito Place
3                          Bus Station
4                                 Park
5                             Beer Bar
6                               Office
7                                Hotel
8                                 Bank
9                       Sandwich Place
10                                 ATM
11                      Breakfast Spot
12                   Mobile Phone Shop
13                 American Restaurant
14                           Hotel Bar
15                                 Bar
16                         Pizza Place
17                Gym / Fitness Center
18                         Pizza Place
19                      Sandwich Place
20                              Office
21                 American Restaurant
22                    Dentist's Office
23                      Sandwich Place
24                            Building
25                       

In [12]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

df = dataframe_ColumbusCity_filtered

df['New_categories'] = pd.np.where(df.categories.str.contains("Food"), "Restaurant",
                   pd.np.where(df.categories.str.contains("Restaurant"), "Restaurant",
                   pd.np.where(df.categories.str.contains("Burrito"), "Restaurant",
                    pd.np.where(df.categories.str.contains("Cuisine"), "Restaurant",
                    pd.np.where(df.categories.str.contains("Pizza"), "Restaurant",
                    pd.np.where(df.categories.str.contains("Cafe"), "Restaurant",
                    pd.np.where(df.categories.str.contains("Sandwich"), "Restaurant",        
                    pd.np.where(df.categories.str.contains("restaurant"), "Restaurant",
                    pd.np.where(df.categories.str.contains("Coffee"), "Restaurant",
                   pd.np.where(df.categories.str.contains("Truck"), "Restaurant", df['categories']))))))))))
df['New_categories']

0                        Restaurant
1                      Intersection
2                        Restaurant
3                       Bus Station
4                              Park
5                          Beer Bar
6                            Office
7                             Hotel
8                              Bank
9                        Restaurant
10                              ATM
11                   Breakfast Spot
12                Mobile Phone Shop
13                       Restaurant
14                        Hotel Bar
15                              Bar
16                       Restaurant
17             Gym / Fitness Center
18                       Restaurant
19                       Restaurant
20                           Office
21                       Restaurant
22                 Dentist's Office
23                       Restaurant
24                         Building
25                       Restaurant
26                       Restaurant
27                         B

In [13]:
from scipy.stats import mode
from numpy import nan

df['New_categories'].value_counts()[0:10]


Restaurant             22
Office                 18
Building               10
None                   10
Government Building     8
Bank                    4
Lawyer                  3
Bus Line                3
Doctor's Office         3
Coworking Space         2
Name: New_categories, dtype: int64

### Cleaned dataset

In [14]:
df.head(50)


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,New_categories
0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"['2 E Broad St (North High St)', 'Columbus, OH...",nan,4e0e63567d8bb178a8a8e0b6,Restaurant
1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"['Broad Street (High street)', 'Columbus, OH 4...",nan,4dd6c521d22d38ef42d7e74f,Intersection
2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"['1 S High St (at Broad St.)', 'Columbus, OH 4...",nan,4ba25a3ef964a5203bf037e3,Restaurant
3,COTA Bus Stop @ Broad & High,Bus Station,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"['Broad Street (at High Street)', 'Columbus, O...",nan,4cb8c2e586dd9521a5abaae6,Bus Station
4,Capitol Square,Park,1 Capitol Sq,NaN,39.960905,-82.999606,"[{'label': 'display', 'lat': 39.9609053407906,...",177,43215,US,Columbus,OH,United States,"['1 Capitol Sq', 'Columbus, OH 43215', 'United...",nan,4caf6efaef1b370464743200,Park
5,The Daily Growler,Beer Bar,1500 Polaris Pkwy,NaN,40.145624,-82.982058,"[{'label': 'display', 'lat': 40.14562420000001...",20473,43240,US,Columbus,OH,United States,"['1500 Polaris Pkwy', 'Columbus, OH 43240', 'U...",nan,5c76d4841cf2e10039e29680,Beer Bar
6,Crowe Horwath LLP,Office,155 W Nationwide Blvd,NaN,39.962236,-83.000652,"[{'label': 'display', 'lat': 39.96223647726225...",5,43215,US,Columbus,OH,United States,"['155 W Nationwide Blvd', 'Columbus, OH 43215'...",nan,4fe85e7ae4b0117429aa1c41,Office
7,"Hotel LeVeque, Autograph Collection",Hotel,50 W Broad St,Broad and Front,39.962299,-83.002059,"[{'label': 'display', 'lat': 39.96229935639491...",115,43215,US,Columbus,OH,United States,"['50 W Broad St (Broad and Front)', 'Columbus,...",nan,58c96261735a4d6c4d5595a3,Hotel
8,Huntington Bank,Bank,17 South High Street,NaN,39.961746,-83.000966,"[{'label': 'display', 'lat': 39.96174568, 'lng...",61,43215,US,Columbus,OH,United States,"['17 South High Street', 'Columbus, OH 43215',...",nan,4b0aef07f964a5204a2a23e3,Bank
9,Jimmy John's,Sandwich Place,20 S Front St,W Capital St,39.961554,-83.002333,"[{'label': 'display', 'lat': 39.96155426911419...",159,43215,US,Columbus,OH,United States,"['20 S Front St (W Capital St)', 'Columbus, OH...",nan,4dc17ee9ae60533f34f34667,Restaurant


## Segmentation & Clustering

#### Collecting data from all sources

In [3]:
df_Columbus = pd.read_csv("dataframe_ColumbusCity_filtered.csv")
print(df_Columbus.shape)

df_Austin = pd.read_csv("dataframe_AustinCity_filtered.csv")
print(df_Austin.shape)

df_Boston = pd.read_csv("dataframe_BostonCity_filtered.csv")
print(df_Boston.shape)

df_Charlotte = pd.read_csv("dataframe_CharlotteCity_filtered.csv")
print(df_Charlotte.shape)

(124, 18)
(120, 18)
(124, 18)
(120, 18)


In [4]:
frames = [df_Columbus, df_Austin, df_Boston, df_Charlotte]
df_result = pd.concat(frames)
df_result.head(5)

,Unnamed: 0,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,New_categories
0,0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"['2 E Broad St (North High St)', 'Columbus, OH...",NaN,4e0e63567d8bb178a8a8e0b6,Restaurant
1,1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"['Broad Street (High street)', 'Columbus, OH 4...",NaN,4dd6c521d22d38ef42d7e74f,Intersection
2,2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"['1 S High St (at Broad St.)', 'Columbus, OH 4...",NaN,4ba25a3ef964a5203bf037e3,Restaurant
3,3,Huntington Bank,Bank,17 South High Street,NaN,39.961746,-83.000966,"[{'label': 'display', 'lat': 39.96174568, 'lng...",61,43215,US,Columbus,OH,United States,"['17 South High Street', 'Columbus, OH 43215',...",NaN,4b0aef07f964a5204a2a23e3,Bank
4,4,COTA Bus Stop @ Broad & High,Bus Station,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"['Broad Street (at High Street)', 'Columbus, O...",NaN,4cb8c2e586dd9521a5abaae6,Bus Station


In [5]:
print('There are {} uniques categories.'.format(len(df_result['categories'].unique())))

There are 142 uniques categories.


In [ ]:
# one hot encoding
df_result_onehot = pd.get_dummies(df_result[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_result_onehot['city'] = df_result['city'] 

# move neighborhood column to the first column
fixed_columns = [df_result_onehot.columns[-1]] + list(df_result_onehot.columns[:-1])
df_result_onehot = df_result_onehot[fixed_columns]

df_result_onehot

In [7]:
df_result_onehot.shape

(488, 143)

In [8]:
df_result_grouped = df_result_onehot.groupby('city').mean().reset_index()
df_result_grouped

,city,ATM,Advertising Agency,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bank,Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bistro,Breakfast Spot,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Café,Capitol Building,Caribbean Restaurant,Casino,Chinese Restaurant,Chiropractor,Church,Circus,City Hall,Clothing Store,Coffee Shop,College Administrative Building,College Arts Building,College Gym,College Lab,Concert Hall,Conference Room,Convenience Store,Convention Center,Courthouse,Coworking Space,Credit Union,Cupcake Shop,Deli / Bodega,Dentist's Office,Design Studio,Dessert Shop,Diner,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Event Space,Eye Doctor,Farmers Market,Financial or Legal Service,Food Truck,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Government Building,Greek Restaurant,Grocery Store,Gym / Fitness Center,High School,Historic Site,History Museum,Hot Spring,Hotel,Hotel Bar,IT Services,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lawyer,Lighthouse,Locksmith,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Non-Profit,None,Nursery School,Office,Optical Shop,Other Great Outdoors,Park,Parking,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Professional & Other Places,Public Art,Radio Station,Record Shop,Restaurant,Roller Rink,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Southern / Soul Food Restaurant,Speakeasy,Steakhouse,Street Food Gathering,Student Center,Sushi Restaurant,Tanning Salon,Tea Room,Tech Startup,Theater,Theme Park,Tourist Information Center,Trail,Wedding Hall,Wine Bar
0,Ausin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Austin,0.008547,0.017094,0.017094,0.000000,0.025641,0.000,0.000000,0.0,0.000000,0.000000,0.017094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.094017,0.000000,0.000000,0.025641,0.017094,0.051282,0.008547,0.008547,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.008547,0.008547,0.008547,0.008547,0.000000,0.008547,0.000000,0.000000,0.017094,0.017094,0.000000,0.000000,0.000000,0.000000,0.008547,0.000000,0.000000,0.000000,0.017094,0.000000,0.008547,0.000000,0.000000,0.000000,0.000000,0.017094,0.008547,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,0.008547,0.008547,0.0,0.000000,0.008547,0.000000,0.008547,0.000000,0.017094,0.000000,0.008547,0.000000,0.000000,0.000000,0.008547,0.000000,0.008547,0.000000,0.000000,0.

In [9]:
df_result_grouped.shape

(10, 143)

In [ ]:
num_top_venues = 5

for hood in df_result_grouped['city']:
    print("----"+hood+"----")
    temp = df_result_grouped[df_result_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [12]:
print(df_result_grouped.shape)

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_result_venues_sorted = pd.DataFrame(columns=columns)
df_result_venues_sorted['city'] = df_result_grouped['city']

for ind in np.arange(df_result_grouped.shape[0]):
    df_result_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_result_grouped.iloc[ind, :], num_top_venues)

df_result_venues_sorted.shape

(10, 143)


(10, 11)

In [13]:
# set number of clusters
kclusters = 4

df_result_grouped_clustering = df_result_grouped.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_result_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 0, 2, 4, 1, 0], dtype=int32)

In [14]:
# add clustering labels
df_result_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_result_merged = df_result

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_result_merged = df_result_merged.join(df_result_venues_sorted.set_index('city'), on='city')

df_result_merged.head() # check the last columns!


,Unnamed: 0,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,New_categories,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"['2 E Broad St (North High St)', 'Columbus, OH...",NaN,4e0e63567d8bb178a8a8e0b6,Restaurant,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
1,1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"['Broad Street (High street)', 'Columbus, OH 4...",NaN,4dd6c521d22d38ef42d7e74f,Intersection,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
2,2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"['1 S High St (at Broad St.)', 'Columbus, OH 4...",NaN,4ba25a3ef964a5203bf037e3,Restaurant,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
3,3,Huntington Bank,Bank,17 South High Street,NaN,39.961746,-83.000966,"[{'label': 'display', 'lat': 39.96174568, 'lng...",61,43215,US,Columbus,OH,United States,"['17 South High Street', 'Columbus, OH 43215',...",NaN,4b0aef07f964a5204a2a23e3,Bank,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
4,4,COTA Bus Stop @ Broad & High,Bus Station,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"['Broad Street (at High Street)', 'Columbus, O...",NaN,4cb8c2e586dd9521a5abaae6,Bus Station,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line


In [15]:
df = pd.DataFrame(df_result_merged)
df.columns

Index(['Unnamed: 0', 'name', 'categories', 'address', 'crossStreet', 'lat',
       'lng', 'labeledLatLngs', 'distance', 'postalCode', 'cc', 'city',
       'state', 'country', 'formattedAddress', 'neighborhood', 'id',
       'New_categories', 'Cluster Labels', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

In [16]:
df.head()

,Unnamed: 0,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,New_categories,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"['2 E Broad St (North High St)', 'Columbus, OH...",NaN,4e0e63567d8bb178a8a8e0b6,Restaurant,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
1,1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"['Broad Street (High street)', 'Columbus, OH 4...",NaN,4dd6c521d22d38ef42d7e74f,Intersection,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
2,2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"['1 S High St (at Broad St.)', 'Columbus, OH 4...",NaN,4ba25a3ef964a5203bf037e3,Restaurant,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
3,3,Huntington Bank,Bank,17 South High Street,NaN,39.961746,-83.000966,"[{'label': 'display', 'lat': 39.96174568, 'lng...",61,43215,US,Columbus,OH,United States,"['17 South High Street', 'Columbus, OH 43215',...",NaN,4b0aef07f964a5204a2a23e3,Bank,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
4,4,COTA Bus Stop @ Broad & High,Bus Station,Broad Street,at High Street,39.961986,-83.000442,"[{'label': 'display', 'lat': 39.96198624401962...",37,43215,US,Columbus,OH,United States,"['Broad Street (at High Street)', 'Columbus, O...",NaN,4cb8c2e586dd9521a5abaae6,Bus Station,0.0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line


In [17]:
df["Cluster Labels"] = df["Cluster Labels"].fillna(0).astype(int)


df.dtypes


Unnamed: 0                  int64
name                       object
categories                 object
address                    object
crossStreet                object
lat                       float64
lng                       float64
labeledLatLngs             object
distance                    int64
postalCode                 object
cc                         object
city                       object
state                      object
country                    object
formattedAddress           object
neighborhood               object
id                         object
New_categories             object
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [ ]:
# create map
import folium

map_clusters = folium.Map(location=[latitude[0], longitude[0]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_result_merged['latitude'], df_result_merged['longitude'], df_result_merged['city'], df_result_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
df.head(3)

,Unnamed: 0,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id,New_categories,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Tim Hortons,Coffee Shop,2 E Broad St,North High St,39.962452,-83.000424,"[{'label': 'display', 'lat': 39.9624524319083,...",32,43215,US,Columbus,OH,United States,"['2 E Broad St (North High St)', 'Columbus, OH...",NaN,4e0e63567d8bb178a8a8e0b6,Restaurant,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
1,1,Broad & High,Intersection,Broad Street,High street,39.962426,-83.000680,"[{'label': 'display', 'lat': 39.96242620803873...",18,43215,US,Columbus,OH,United States,"['Broad Street (High street)', 'Columbus, OH 4...",NaN,4dd6c521d22d38ef42d7e74f,Intersection,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line
2,2,Cinco Burritos & Tacos,Burrito Place,1 S High St,at Broad St.,39.962214,-83.000839,"[{'label': 'display', 'lat': 39.96221394279797...",12,43215,US,Columbus,OH,United States,"['1 S High St (at Broad St.)', 'Columbus, OH 4...",NaN,4ba25a3ef964a5203bf037e3,Restaurant,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line


In [37]:
df = df[['state','city','lat','lng', 'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue', 'name', 'categories', 'New_categories']]
df.head()

,state,city,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,name,categories,New_categories
0,OH,Columbus,39.962452,-83.000424,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line,Tim Hortons,Coffee Shop,Restaurant
1,OH,Columbus,39.962426,-83.000680,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line,Broad & High,Intersection,Intersection
2,OH,Columbus,39.962214,-83.000839,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line,Cinco Burritos & Tacos,Burrito Place,Restaurant
3,OH,Columbus,39.961746,-83.000966,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line,Huntington Bank,Bank,Bank
4,OH,Columbus,39.961986,-83.000442,0,Office,None,Building,Government Building,Food Truck,Bank,Sandwich Place,Pizza Place,Lawyer,Bus Line,COTA Bus Stop @ Broad & High,Bus Station,Bus Station


In [ ]:
#Cluster 1
df.loc[df['Cluster Labels'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]]

In [39]:
#Cluster 2
df.loc[df['Cluster Labels'] == 1, df.columns[[1] + list(range(5, df.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,name,categories,New_categories
87,Santa Barbara,Miscellaneous Shop,Courthouse,Design Studio,Dentist's Office,Deli / Bodega,Cupcake Shop,Credit Union,Coworking Space,Wine Bar,Dessert Shop,The Kitchen Company Santa Barbara,Miscellaneous Shop,Miscellaneous Shop


In [40]:
#Cluster 3
df.loc[df['Cluster Labels'] == 2, df.columns[[1] + list(range(5, df.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,name,categories,New_categories
3,Matthews,Hotel,Wine Bar,Courthouse,Design Studio,Dentist's Office,Deli / Bodega,Cupcake Shop,Credit Union,Coworking Space,Convention Center,Hampton Inn Charlotte/Matthews,Hotel,Hotel


In [41]:
#Cluster 4
df.loc[df['Cluster Labels'] == 3, df.columns[[1] + list(range(5, df.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,name,categories,New_categories
28,Ausin,Chiropractor,Wine Bar,Courthouse,Design Studio,Dentist's Office,Deli / Bodega,Cupcake Shop,Credit Union,Coworking Space,Convention Center,San Jacinto Chiropractic,Chiropractor,Chiropractor


In [42]:
#Cluster 4
df.loc[df['Cluster Labels'] == 4, df.columns[[1] + list(range(5, df.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,name,categories,New_categories
92,Pekanbaru,High School,Courthouse,Design Studio,Dentist's Office,Deli / Bodega,Cupcake Shop,Credit Union,Coworking Space,Convention Center,Diner,SMA Cendana,High School,High School
